In [1]:
import sys
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 
%pylab inline
from IPython.core.display import clear_output
import os,shutil
import re
import copy as cp
from scipy import stats
import subprocess
import glob
import pandas as pd
import pmx
from pmx import *
from pmx.parser import *
from pmx.forcefield import *
from pmx.utils import create_folder
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem

Populating the interactive namespace from numpy and matplotlib


In [2]:
# set parameters
def clean_up( path ):
    # clean up
    toclean = glob.glob('{0}/*#'.format(path))
    for clean in toclean:
        os.remove(clean)
        
def clean_pdb( infile, outfile ):
    fp = open(infile,'r')
    lines = fp.readlines()
    fp.close()
    
    fp = open(outfile,'w')
    for l in lines:
        if ('ATOM' in l) or ('HETATM' in l):
            fp.write(l)
    fp.close()
            
ff = 'charmm'

rawpath = '/netmount/glic3/vgapsys/blvrb/data/raw_pdb_all'
struct_top = '/netmount/glic3/vgapsys/blvrb/struct_top'
if ff=='charmm':
    struct_top = '/netmount/glic3/vgapsys/blvrb/charmm_struct_top'
ligs = ['ppa','fmg','icl','ptc','oss','tei','sas','a80','apo']
sites = ['a','b']

--------------------
1. Separate xray structures: protein, ligands, water
--------------------

In [5]:
for lig in ligs:
    for site in sites:
        
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue
        
        initpdb = '{0}/{1}_mol_{2}.pdb'.format(rawpath,lig,site)
        
        folder = '{0}/{1}_{2}'.format(struct_top,lig,site)
        create_folder(folder)
        os.chdir(folder)
        
        # protein
        folder = '{0}/{1}_{2}/protein'.format(struct_top,lig,site)
        create_folder(folder)
        os.chdir(folder)
        case = 'Protein'
        cmd = 'echo {0} | gmx trjconv -s {1} -f {1} -o protein.pdb'.format(case,initpdb)
        os.system(cmd)
        clean_up(folder)
        
        # water
        folder = '{0}/{1}_{2}/water'.format(struct_top,lig,site)
        create_folder(folder)
        os.chdir(folder)
        case = 'SOL'
        cmd = 'echo {0} | gmx trjconv -s {1} -f {1} -o water.pdb'.format(case,initpdb)
        os.system(cmd)
        clean_up(folder)        
        
        # ligand
        folder = '{0}/{1}_{2}/ligand'.format(struct_top,lig,site)
        create_folder(folder)
        os.chdir(folder)
        case = lig.upper()
        cmd = 'echo {0} | gmx trjconv -s {1} -f {1} -o mol.pdb'.format(case,initpdb)
        os.system(cmd)
        clean_up(folder)        
        
        # nad
        folder = '{0}/{1}_{2}/nad'.format(struct_top,lig,site)
        create_folder(folder)
        os.chdir(folder)
        case = 'NAP'
        cmd = 'echo {0} | gmx trjconv -s {1} -f {1} -o nad.pdb'.format(case,initpdb)
        os.system(cmd)
        clean_up(folder)                
        
#        print(lig,site,initpdb)

----------------------
2a. Topologies protein
---------------------

In [7]:
for lig in ligs:
    for site in sites:
        
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue        
        
        # protein
        folder = '{0}/{1}_{2}/protein'.format(struct_top,lig,site)
        os.chdir(folder)
        cmd = 'gmx pdb2gmx -f protein.pdb -o protein.pdb -ff amber99sb-star-ildn -water tip3p -i posre_protein.itp'
        if ff=='charmm':
            cmd = 'gmx pdb2gmx -f protein.pdb -o protein.pdb -ff charmm36-jul2020 -water tip3p -i posre_protein.itp'
        os.system(cmd)
        
        # clean pdb
        clean_pdb( 'protein.pdb', 'protein.pdb')
        
        # top2itp
        cmd = 'python ~/scripts/ligand_scripts/python3/top2itp.py -p topol.top -oitp protein.itp'
        os.system(cmd)
        
        clean_up(folder)

----------------------
2b. Structures water
---------------------

In [8]:
for lig in ligs:
    for site in sites:
        
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue        
        
        # water
        folder = '{0}/{1}_{2}/water'.format(struct_top,lig,site)
        os.chdir(folder)
        cmd = 'gmx pdb2gmx -f water.pdb -o water.pdb -ff amber99sb-star-ildn -water tip3p -i posre.itp'
        if ff=='charmm':
            cmd = 'gmx pdb2gmx -f water.pdb -o water.pdb -ff charmm36-jul2020 -water tip3p -i posre.itp'
    
        os.system(cmd)
        
        # clean pdb
        clean_pdb( 'water.pdb', 'water.pdb')
        
        clean_up(folder)

----------------------
2c. Topologies ligand
---------------------

In [69]:
#m = Chem.MolFromSmiles('C1(=NCC=CC1)NS(=O)(=O)c1ccc(cc1)/N=N/c1cc(c(cc1)O)C(=O)O')
#m = Chem.AddHs(m)
#AllChem.EmbedMolecule(m)
#print(Chem.MolToMolBlock(m),file=open('/netmount/glic3/vgapsys/blvrb/struct_top/foo.mol','w+'))

In [4]:
def create_g09_jobscript(fname,nproc=8,jobname='lig'):
    fp = open(fname,'w')
    fp.write('#!/bin/bash\n')
    fp.write('#$ -S /bin/bash\n')
    fp.write('#$ -pe openmp_fast {0}\n'.format(nproc))
    fp.write('#$ -q *\n')
    fp.write('#$ -N {0}\n'.format(jobname))
    fp.write('#$ -M vgapsys@gwdg.de\n')
    fp.write('#$ -m n\n')
    fp.write('#$ -l h_rt=24:00:00\n')
    fp.write('#$ -l h_data=2000M\n')    
    fp.write('#$ -l ds=1\n')
    fp.write('#$ -cwd\n')
    fp.write('. /etc/profile.d/modules.sh\n')
    fp.write('module load sge\n')
    fp.write('export MPIRUN=mpiexec\n')
    fp.write('export PYTHONPATH=/home/vgapsys/owl/pmx_new/lib64/python\n')
    fp.write('export MPD_CON_EXT="sge_$JOB_ID.$SGE_TASK_ID"\n')
    fp.write('export MPD_TMPDIR=$TMPDIR\n')
    fp.write('export I_MPI_PIN=disable\n')
    fp.write('export g09root=/usr/local/gaussian/g09-d01\n')
    fp.write('source $g09root/g09/bsd/g09.profile\n')
    fp.write('export GAUSS_SCRDIR=$TMPDIR\n')
    fp.write('g09 lig.gau\n')
    fp.close()
    
def change_nproc(fname,nproc=4):
    fp = open(fname,'r')
    lines = fp.readlines()
    fp.close()
    
    ll = '%Nproc={0}\n'.format(nproc)
    fp = open(fname,'w')
    for l in lines:
        if 'Nproc' in l:
            continue
        fp.write(l)
        if 'chk' in l:
            fp.write(ll)
    fp.close()
    
def remove_g09_opt(fname):
    fp = open(fname,'r')
    lines = fp.readlines()
    fp.close()
    
    fp = open(fname,'w')
    for l in lines:
        if l.startswith('#'):
            foo = l.rstrip().split()
            newl = ''
            for el in foo:
                if 'opt' in el:
                    continue
                newl+= '{0} '.format(el)
            l = '{0}\n'.format(newl)
        fp.write(l)
    fp.close()    
    
def remove_hydrogen( fname, atomNum=30 ):
    m = Model(fname)
    newm = Model()
    for a in m.atoms:
        if a.id==atomNum:
            continue
        newm.atoms.append(a)
    newm.write(fname)
            

In [72]:
###################################
######## GAFF #####################
###################################

for lig in ligs:
    if 'apo' in lig:
        continue
        
    for site in sites:
        
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue        
        
        # add hydrogens
        folder = '{0}/{1}_{2}/ligand'.format(struct_top,lig,site)
        os.chdir(folder)
        cmd = 'babel -h mol.pdb mol.pdb -p 6.5'
        os.system(cmd)
        cmd = 'babel -h mol.pdb mol.sdf -p 6.5'
        os.system(cmd)               
        
        if ('sas' in lig) and site=='a': # some ligands are prepared wrongly
            cmd = 'cp {0}/manual_str {0}/mol.pdb'.format(folder)
            os.system(cmd)
            cmd = 'babel mol.pdb mol.sdf'
            os.system(cmd)
                    
        # print out the charge
        mol = Chem.MolFromMolFile('mol.sdf')
        
        if 'ptc' in lig:
            charge = -1
            remove_hydrogen( 'mol.pdb', atomNum=30 ) # for ptc ligand babel assigns wrong protonation        
        else:
            try:
                charge = rdkit.Chem.rdmolops.GetFormalCharge(mol)
                if 'sas' in lig:
                    charge = -1            
                print(lig,site,charge)
            except: # ptc
                print(lig,site,charge,'problems')
        
        # gaussian input
        folder = '{0}/{1}_{2}/ligand/gaussian'.format(struct_top,lig,site)
        create_folder(folder)
        os.chdir(folder)
        cmd = 'antechamber -fi pdb -fo gcrt -i ../mol.pdb -o lig.gau -nc {0}'.format(charge)
        os.system(cmd)
        remove_g09_opt('lig.gau')
        
        # change processors
        change_nproc('lig.gau',nproc=8)
        
        # jobscript
        create_g09_jobscript('jobscript',nproc=8,jobname='lig')
               
        clean_up(folder)
        
# create job submission script
os.chdir(struct_top)
fp = open('submit.sh','w')
for lig in ligs:
    for site in sites:
        l = 'cd {0}/{1}_{2}/ligand/gaussian\nqsub jobscript\n\n'.format(struct_top,lig,site)
        regexp = re.compile('netmount')
        l = regexp.sub('home',l)
        fp.write(l)
fp.close()

ppa a -2
ppa b -2
fmg a -1
fmg b -1
icl a -1
ptc a -1
ptc b -1
oss a -2
oss b -2
tei a -1
tei b -1


RDKit ERROR: [19:15:15] Explicit valence for atom # 0 C greater than permitted


sas a -1 problems
sas b -1
a80 a -1


In [73]:
######################################
##### run gaussian on the cluster ####
######################################

In [74]:
### Topologies ###
        
for lig in ligs:
    if 'apo' in lig:
        continue
        
    for site in sites:
        
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue        
        
        folder = '{0}/{1}_{2}/ligand/top'.format(struct_top,lig,site)
        create_folder(folder)
        os.chdir(folder)        
        cmd = 'python ~/scripts/ligand_scripts/python3/sigmahole.py \
               -ff gaff2 -log ../gaussian/lig.log \
               -opdb ligand.pdb -oitp ../ligand.itp -offitp ../ffligand.itp -nosh'        
        os.system(cmd)
        
        # use the initial mol.pdb as the ligand structure
#        cmd = 'cp ../mol.pdb ../ligand.pdb'
#        os.system(cmd)

        # (create hybrid topologies)
#        itpfile = '{0}/{1}_{2}/ligand/ligand.itp'.format(struct_top,lig,site)
#        itpfileA = '{0}/{1}_{2}/ligand/ligandA.itp'.format(struct_top,lig,site)
#        itpfileB = '{0}/{1}_{2}/ligand/ligandB.itp'.format(struct_top,lig,site)
#        ffitpfile = '{0}/{1}_{2}/ligand/ffligand.itp'.format(struct_top,lig,site)               
#        TOPdecouple( itpfile=itpfile, itpfileA=itpfileA, itpfileB=itpfileB, ffitpfile=ffitpfile )
        
        # add posre
        cmd = 'python /home/vgapsys/scripts/misc_python3_scripts/free_energy_setup_python3/script_position_restraints.py \
              -i ../ligand.itp -o ../ligand.itp -allHeavy -ifdef'
        os.system(cmd)
        cmd = 'python /home/vgapsys/scripts/misc_python3_scripts/free_energy_setup_python3/script_position_restraints.py \
              -i ../ligandA.itp -o ../ligandA.itp -allHeavy -ifdef'
        os.system(cmd)
        cmd = 'python /home/vgapsys/scripts/misc_python3_scripts/free_energy_setup_python3/script_position_restraints.py \
              -i ../ligandB.itp -o ../ligandB.itp -allHeavy -ifdef'
        os.system(cmd)
        
        clean_up(folder)


In [12]:
struct_top

'/netmount/glic3/vgapsys/blvrb/charmm_struct_top'

In [25]:
###################################
######## CGENFF ###################
###################################
# Note: for tei_a and tei_b need to manually comment out two dihedrals:
#    17      3     15     14    9
#    17      3     15     16    9
# these dihedrals do not appear in the topology when paramchem webserver is used

%env SILCSBIODIR=/home/vgapsys/scripts/cgenff/silcsbio.2020.2
%env GMXLIB=/netmount/glic3/vgapsys/solvation_dG_freeSolv/ff
scriptpath = '/home/vgapsys/scripts/ligand_scripts/python3'

if ff=='charmm':
    for lig in ligs:
        if 'apo' in lig:
            continue

        for site in sites:

            if lig=='icl' or lig=='a80':
                if site=='b':
                    continue        

            # copy the structures from gaff folder
            cmd = 'cp /netmount/glic3/vgapsys/blvrb/struct_top/{0}_{1}/ligand/mol.pdb \
                  {2}/{0}_{1}/ligand/.'.format(lig,site,struct_top)
#            os.system(cmd)
            
            # create mol2
            cmd = 'babel {0}/{1}_{2}/ligand/mol.pdb \
                  {0}/{1}_{2}/ligand/mol.mol2'.format(struct_top,lig,site)
#            os.system(cmd)

            # run cgenff
            os.chdir('{0}/{1}_{2}/ligand'.format(struct_top,lig,site))
            cmd = '/home/vgapsys/scripts/cgenff/silcsbio.2020.2/cgenff/cgenff_to_gmx.sh \
                  mol=mol.mol2'
#            os.system(cmd)    
            
            # now convert to gromacs format
            # 1. top2itp
            cmd = 'python ~/scripts/ligand_scripts/python3/top2itp.py -p mol_gmx.top \
                  -oitp initMOL.itp -name MOL'
#            os.system(cmd)

            # 2. fill parameters
            ffpath = '/netmount/glic3/vgapsys/solvation_dG_freeSolv/ff/charmm36-jul2020.ff'
            cmd = 'python {0}/fill_cgenff_itp.py -itp initMOL.itp \
                  -ff {1} -o MOL.itp -prm charmm36.ff/LIG_ffbonded.itp'\
                 .format(scriptpath,ffpath)
#            os.system(cmd)
            cmd = 'mv MOL.itp ligand.itp'
#            os.system(cmd)
            cmd = 'mv mol_gmx.pdb mol.pdb'
#            os.system(cmd)            

            # add posre
            folder = '{0}/{1}_{2}/ligand'.format(struct_top,lig,site)
            cmd = 'python /home/vgapsys/scripts/misc_python3_scripts/free_energy_setup_python3/script_position_restraints.py \
                  -i {0}/ligand.itp -o {0}/ligand.itp -allHeavy -ifdef'.format(folder)
            os.system(cmd)
        
            clean_up(folder)

env: SILCSBIODIR=/home/vgapsys/scripts/cgenff/silcsbio.2020.2
env: GMXLIB=/netmount/glic3/vgapsys/solvation_dG_freeSolv/ff


----------------------
2d. Structures nadp and one topology
---------------------

In [55]:
# in case it is needed, extract nad.pdb again

for lig in ligs:
    for site in sites:
        
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue
        
        initpdb = '{0}/{1}_mol_{2}.pdb'.format(rawpath,lig,site)
        
        folder = '{0}/{1}_{2}'.format(struct_top,lig,site)
        create_folder(folder)
        os.chdir(folder)
        
     # nad
        folder = '{0}/{1}_{2}/nad'.format(struct_top,lig,site)
        create_folder(folder)
        os.chdir(folder)
        case = 'NAP'
        cmd = 'echo {0} | gmx trjconv -s {1} -f {1} -o nad.pdb'.format(case,initpdb)
        os.system(cmd)
        clean_up(folder)                
        
#        print(lig,site,initpdb)

In [56]:
# structures
hydrogensToRemove = {'a80_a':55, 'sas_a':55, 'sas_b':66, 'tei_a':55, 'tei_b':66, 'oss_a':55, 'oss_b':54,
                    'ptc_a':66, 'ptc_b':55, 'icl_a':66, 'fmg_a':55, 'fmg_b':54, 'ppa_a':66, 'ppa_b':54,
                    'apo_a':55, 'apo_b':65}

for lig in ligs:
        
    for site in sites:
        
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue        
        
        # add hydrogens
        folder = '{0}/{1}_{2}/nad'.format(struct_top,lig,site)
        os.chdir(folder)
        cmd = 'babel nad.pdb nad.sdf'
        os.system(cmd)               
        cmd = 'babel -h -p 6.5 nad.pdb nad.pdb'
        os.system(cmd)
        
        # babel doesn't do well, so need to remove some hydrogens
        key = '{0}_{1}'.format(lig,site)
        remove_hydrogen( 'nad.pdb', atomNum=hydrogensToRemove[key] )
                  
#        clean_up(folder)

In [85]:
########### 
# manually adjust the structures, because babel is very inconsistent
###########

In [8]:
# topology
folder = '{0}/top_nad'.format(struct_top)
create_folder(folder)
os.chdir(folder)

# gaussian input
folder = '{0}/top_nad/gaussian'.format(struct_top)
create_folder(folder)
os.chdir(folder)

charge = -3
cmd = 'antechamber -fi pdb -fo gcrt -i ../manual_nadH.pdb -o lig.gau -nc {0}'.format(charge)
os.system(cmd)
remove_g09_opt('lig.gau')        
        
# change processors
change_nproc('lig.gau',nproc=8)
        
# jobscript
create_g09_jobscript('jobscript',nproc=8,jobname='lig')        


######################################
##### run gaussian on the cluster ####
######################################

In [9]:
### Topology ###
         
folder = '/netmount/glic3/vgapsys/blvrb/struct_top/top_nad/top'
create_folder(folder)
os.chdir(folder)        
cmd = 'python ~/scripts/ligand_scripts/python3/sigmahole.py \
               -ff gaff2 -log ../gaussian/lig.log \
               -opdb ../nad.pdb -oitp ../nad.itp -offitp ../ffnad.itp -nosh -ligname NAD'        
os.system(cmd)

# add posre
cmd = 'python /home/vgapsys/scripts/misc_python3_scripts/free_energy_setup_python3/script_position_restraints.py \
       -i ../nad.itp -o ../nad.itp -allHeavy -ifdef'
os.system(cmd)
        
clean_up(folder)


# place topologies in each folder
for lig in ligs:
    for site in sites:
        
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue        
        
        nadfolder = '/netmount/glic3/vgapsys/blvrb/struct_top/top_nad'
        folder = '{0}/{1}_{2}/nad'.format(struct_top,lig,site)
        # itp
        cmd = 'cp {0}/nad.itp {1}/.'.format(nadfolder,folder)
        os.system(cmd)
        # ffitp
        cmd = 'cp {0}/ffnad.itp {1}/.'.format(nadfolder,folder)
        os.system(cmd)


In [24]:
###################################
######## CGENFF ###################
###################################

%env SILCSBIODIR=/home/vgapsys/scripts/cgenff/silcsbio.2020.2
%env GMXLIB=/netmount/glic3/vgapsys/solvation_dG_freeSolv/ff
scriptpath = '/home/vgapsys/scripts/ligand_scripts/python3'

if ff=='charmm':
    for lig in ligs:

        for site in sites:

            if lig=='icl' or lig=='a80':
                if site=='b':
                    continue                   
            
            # copy the structures from gaff folder
            cmd = 'cp /netmount/glic3/vgapsys/blvrb/struct_top/{0}_{1}/nad/nad.pdb \
                  {2}/{0}_{1}/nad/.'.format(lig,site,struct_top)
#            os.system(cmd)
            
            # create mol2
            cmd = 'babel {0}/{1}_{2}/nad/nad.pdb \
                  {0}/{1}_{2}/nad/nad.mol2'.format(struct_top,lig,site)
#            os.system(cmd)

            # run cgenff
            os.chdir('{0}/{1}_{2}/nad'.format(struct_top,lig,site))
            cmd = '/home/vgapsys/scripts/cgenff/silcsbio.2020.2/cgenff/cgenff_to_gmx.sh \
                  mol=nad.mol2'
#            os.system(cmd)    
            
            # now convert to gromacs format
            # 1. top2itp
            cmd = 'python ~/scripts/ligand_scripts/python3/top2itp.py -p nad_gmx.top \
                  -oitp initNAD.itp -name NAD'
#            os.system(cmd)

            # 2. fill parameters
            ffpath = '/netmount/glic3/vgapsys/solvation_dG_freeSolv/ff/charmm36-jul2020.ff'
            cmd = 'python {0}/fill_cgenff_itp.py -itp initNAD.itp \
                  -ff {1} -o nad.itp -prm charmm36.ff/LIG_ffbonded.itp'\
                 .format(scriptpath,ffpath)
            os.system(cmd)
            cmd = 'mv nad_gmx.pdb nad.pdb'
#            os.system(cmd)                        

            # add posre
            cmd = 'python /home/vgapsys/scripts/misc_python3_scripts/free_energy_setup_python3/script_position_restraints.py \
                  -i {0}/{1}_{2}/nad/nad.itp -o {0}/{1}_{2}/nad/nad.itp -allHeavy -ifdef' \
                  .format(struct_top,lig,site)
            os.system(cmd)
        
            clean_up('{0}/{1}_{2}/nad'.format(struct_top,lig,site))

env: SILCSBIODIR=/home/vgapsys/scripts/cgenff/silcsbio.2020.2
env: GMXLIB=/netmount/glic3/vgapsys/solvation_dG_freeSolv/ff


--------------
NADP+ based on Ryde topology
---------------

Take the topology generated for Ryde: 
/netmount/glic3/vgapsys/blvrb/struct_top/top_nad/top_ryde/nad.itp

Then shuffle nad atoms to match the topology

In [4]:
# atom mapping is already established
pairs = '/netmount/glic3/vgapsys/blvrb/struct_top/top_nad/top_ryde/pairs1.dat'
# from this path take the reference positions for nad atoms of which will be reshuffled
refpath = '/netmount/glic3/vgapsys/blvrb/struct_top'
# topology path
toppath = '/netmount/glic3/vgapsys/blvrb/struct_top/top_nad/top_ryde'

for lig in ['oss','apo']:
    for site in sites:
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue        
                
        outpath = '/netmount/glic3/vgapsys/blvrb/ryde_struct_top/{0}_{1}'.format(lig,site)
        print(outpath)
        cmd = 'python ~/scripts/ligand_scripts/python3/reorder_pdb_atoms.py \
              -target {0}/{1}_{2}/nad/nad.pdb \
              -pairs {3} -o {4}/nad/nad.pdb'.format(refpath,lig,site,pairs,outpath)
        os.system(cmd)
        
        # copy topology
        cmd = 'cp {0}/nad.itp {1}/nad/.'.format(toppath,outpath)
        os.system(cmd)
        cmd = 'cp {0}/ffnad.itp {1}/nad/.'.format(toppath,outpath)
        os.system(cmd)
        
        # posre (already in the topology)
#        cmd = 'echo -e "0 & ! a H*\nq\n" | gmx make_ndx -f {0}/nad/nad.pdb \
#               -o {0}/nad/index.ndx'.format(outpath)
#        os.system(cmd)
#        cmd = 'echo 3 | gmx genrestr -f {0}/nad/nad.pdb -o {0}/nad/posre.itp \
#               -n {0}/nad/index.ndx'.format(outpath)
#        os.system(cmd)
        
        clean_up( '{0}/nad'.format(outpath) )
        



/netmount/glic3/vgapsys/blvrb/ryde_struct_top/oss_a
/netmount/glic3/vgapsys/blvrb/ryde_struct_top/oss_b
/netmount/glic3/vgapsys/blvrb/ryde_struct_top/apo_a
/netmount/glic3/vgapsys/blvrb/ryde_struct_top/apo_b


Also do the same for NADPH

In [5]:
# atom mapping is already established
pairs = '/netmount/glic3/vgapsys/blvrb/struct_top/top_nad/top_ryde_nadph/pairs1.dat'
# from this path take the reference positions for nad atoms of which will be reshuffled
refpath = '/netmount/glic3/vgapsys/blvrb/struct_top'
# topology path
toppath = '/netmount/glic3/vgapsys/blvrb/struct_top/top_nad/top_ryde_nadph'

for lig in ['oss','apo']:
    for site in sites:
        if lig=='icl' or lig=='a80':
            if site=='b':
                continue           
        
        outpath = '/netmount/glic3/vgapsys/blvrb/ryde_struct_top_nadph/{0}_{1}'.format(lig,site)
        print(outpath)
        
        # add hydrogen
        m = Model('{0}/{1}_{2}/nad/nad.pdb'.format(refpath,lig,site))
        newatom = cp.deepcopy(m.atoms[-1])
        newatom.x = [-16.231,3.014,6.547]
        newatom.name = 'H42'
        m.atoms.append(newatom)
        m.write('{0}/nad/tmpnad.pdb'.format(outpath))
        
        cmd = 'python ~/scripts/ligand_scripts/python3/reorder_pdb_atoms.py \
              -target {0}/nad/tmpnad.pdb \
              -pairs {1} -o {0}/nad/nad.pdb'.format(outpath,pairs)
        os.system(cmd)
        
        # copy topology
        cmd = 'cp {0}/nad.itp {1}/nad/.'.format(toppath,outpath)
        os.system(cmd)
        cmd = 'cp {0}/ffnad.itp {1}/nad/.'.format(toppath,outpath)
        os.system(cmd)

        
        clean_up( '{0}/nad'.format(outpath) )       


/netmount/glic3/vgapsys/blvrb/ryde_struct_top_nadph/oss_a
/netmount/glic3/vgapsys/blvrb/ryde_struct_top_nadph/oss_b
/netmount/glic3/vgapsys/blvrb/ryde_struct_top_nadph/apo_a
/netmount/glic3/vgapsys/blvrb/ryde_struct_top_nadph/apo_b
